# MYSQL Database connector

Before running this notebook make sure the MySQL Python Connector is installed 
TODO: Add an examples of how to install it

## Imports

In [1]:
import pydough

## Loading credencials and connecting to MySQL

In [2]:
mysql_username = "root"
mysql_password = "root"
mysql_tpch_db = "tpch_db"
mysql_host = "127.0.0.1" # I think is the default value (localhost)

pydough.active_session.load_metadata_graph("../metadata/tpch_demo_graph.json", "TPCH")
pydough.active_session.connect_database("mysql", 
        user=mysql_username,
        password=mysql_password,
        database=mysql_tpch_db,
        host=mysql_host,
)

DatabaseContext(connection=<pydough.database_connectors.database_connector.DatabaseConnection object at 0x10e395bd0>, dialect=<DatabaseDialect.MYSQL: 'mysql'>)

In [3]:
%load_ext pydough.jupyter_extensions

In [4]:
%%pydough
# TPCH Q1
output = ( customers.CALCULATE(
    name
).TOP_K(5, by=name)
)

pydough.to_df(output)

ERROR WHILE EXECUTING QUERY:
SELECT
  c_name AS name
FROM main.CUSTOMER
ORDER BY
  c_name
LIMIT 5


ProgrammingError: 1049 (42000): Unknown database 'main'